In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

In [2]:
iris = load_iris()
X = iris['data']
y = iris['target']
clf = DecisionTreeClassifier(random_state=0)
clf.fit(X, y)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best')

In [3]:
left_nodes = clf.tree_.children_left[clf.tree_.children_left>0]
right_nodes = clf.tree_.children_right[clf.tree_.children_right>0]
node_indicator = clf.decision_path(X)
path_list = []
for i, j in enumerate(X):
    path_list.append(node_indicator.indices[node_indicator.indptr[i]:node_indicator.indptr[i+1]])

## Convert path to strings
path_column = np.array([])
for i, j in enumerate(X):
    path_as_string = []
    for node in path_list[i]:
        if node == 0:
            path_as_string.append('S')
        elif node in left_nodes:
            path_as_string.append('L')
        elif node in right_nodes:
            path_as_string.append('R')
            
    path_as_string.append('E')
    path_as_string = ' '.join(path_as_string)
    path_column = np.append(path_column, path_as_string)

In [4]:
chars = ['S', 'L', 'R', 'E']
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

Xnew = np.hstack((X, path_column.reshape(-1,1)))
path_sequence = Xnew[:,4]
data = pd.DataFrame(Xnew)
data[5]=y
df = data.sample(frac=1).reset_index(drop=True)

# prepare dataset for training
get_path_lengths = lambda t: len(t.split())
paths_lengths = np.array([get_path_lengths(xi) for xi in path_sequence])

In [5]:
vocab_size = 4
label_size = 3
feature_size = 4
# cut the text in semi-redundant sequences of maxlen characters
maxlen = np.max(paths_lengths)
sentences = []
next_chars = []
features = []
labels = []

for i in range(0, len(df)):
    # get the feature
    curr_feat = np.array([df.iloc[i,0:4]])
    curr_path = df.iloc[i,4].split()
    curr_path_len = len(curr_path)
    curr_label = y[i]
    for j in range(1,curr_path_len):
        features.append(curr_feat)
        labels.append(curr_label)
        sentences.append(curr_path[0:j])
        next_chars.append(curr_path[j])
print('Vectorization...')

x_sent = np.zeros((len(sentences), maxlen, vocab_size), dtype=np.bool)
x_feat = np.zeros((len(sentences), feature_size), dtype=np.float)
y_chars = np.zeros((len(sentences), vocab_size), dtype=np.bool)
y_feat = np.zeros((len(sentences), label_size), dtype=np.float)
#from keras.utils import to_categorical
#y_feat_tmp = to_categorical(df[5])

Vectorization...


In [6]:
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x_sent[i, t, char_indices[char]] = 1
    y_chars[i, char_indices[next_chars[i]]] = 1
    x_feat[i,:] = features[i]
    y_feat[i,labels[i]]=1

In [7]:
index = 10
print(y_chars[index],y_feat[index],x_sent[index],x_feat[index])
print(y_chars.shape,y_feat.shape,x_sent.shape,x_feat.shape)

[False False  True False] [1. 0. 0.] [[ True False False False]
 [False False  True False]
 [False False  True False]
 [False  True False False]
 [False False False False]
 [False False False False]
 [False False False False]] [5.9 3.2 4.8 1.8]
(560, 4) (560, 3) (560, 7, 4) (560, 4)


In [9]:
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Concatenate, concatenate, Flatten

h1_size = 5
latent_dim = 5

input_x_features = Input(shape=(feature_size,),name='ip_x')
hidden_state_x = Dense(h1_size, activation='relu',name='hidden_x')(input_x_features)
output_labels = Dense(3, activation='linear',name='op_x')(hidden_state_x)

input_sent_features = Input(shape=(maxlen,vocab_size),name='ip_sent')
decoder = LSTM(latent_dim,return_state=False,return_sequences=False,name='lstm_sent')
decoder_outputs = decoder(input_sent_features)

merge_layer = concatenate([hidden_state_x,decoder_outputs],name='cat')
output_chars = Dense(vocab_size, activation='softmax',name='op_sent')(merge_layer)
model = Model([input_x_features,input_sent_features], [output_labels,output_chars])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
ip_x (InputLayer)               (None, 4)            0                                            
__________________________________________________________________________________________________
ip_sent (InputLayer)            (None, 7, 4)         0                                            
__________________________________________________________________________________________________
hidden_x (Dense)                (None, 5)            25          ip_x[0][0]                       
__________________________________________________________________________________________________
lstm_sent (LSTM)                (None, 5)            200         ip_sent[0][0]                    
_____________________________________

In [13]:
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
model.fit([x_feat,x_sent],[y_feat,y_chars],batch_size =10, epochs = 10,verbose=1)

Epoch 1/10
560/560 [==============================] - 2s 4ms/step - loss: 6.3213 - op_x_loss: 5.3535 - op_sent_loss: 0.9677 - op_x_acc: 0.3304 - op_sent_acc: 0.5625
Epoch 2/10
560/560 [==============================] - 1s 1ms/step - loss: 6.2764 - op_x_loss: 5.3535 - op_sent_loss: 0.9229 - op_x_acc: 0.3339 - op_sent_acc: 0.5821
Epoch 3/10
560/560 [==============================] - 1s 2ms/step - loss: 6.2321 - op_x_loss: 5.3535 - op_sent_loss: 0.8786 - op_x_acc: 0.3339 - op_sent_acc: 0.6036
Epoch 4/10
560/560 [==============================] - 1s 1ms/step - loss: 6.1929 - op_x_loss: 5.3535 - op_sent_loss: 0.8394 - op_x_acc: 0.3339 - op_sent_acc: 0.5696
Epoch 5/10
560/560 [==============================] - 1s 2ms/step - loss: 6.1576 - op_x_loss: 5.3535 - op_sent_loss: 0.8041 - op_x_acc: 0.3339 - op_sent_acc: 0.5964
Epoch 6/10
560/560 [==============================] - 1s 1ms/step - loss: 6.1260 - op_x_loss: 5.3535 - op_sent_loss: 0.7725 - op_x_acc: 0.3339 - op_sent_acc: 0.6250
Epoch 7/10